In [2]:
import torch
from torch import nn
import numpy as np
import pandas as pd
import PIL
import glob
import os
import torchvision
import gym
from pyvirtualdisplay import Display
import matplotlib.pyplot as plt
import copy
import time
import random

In [1]:
DQN=torch.nn.Sequential(nn.Linear(in_features=4,out_features=32,bias=True),
                       nn.ReLU(),
                       nn.Linear(in_features=32,out_features=64,bias=True),
                       nn.ReLU(),
                       nn.Linear(in_features=64,out_features=128,bias=True),
                       nn.ReLU(),
                       nn.Linear(in_features=128,out_features=256,bias=True),
                       nn.ReLU(),
                       nn.Linear(in_features=256,out_features=512,bias=True),
                       nn.ReLU(),
                       nn.Linear(in_features=512,out_features=256,bias=True),
                       nn.ReLU(),
                       nn.Linear(in_features=256,out_features=2,bias=True))
targetDQN=copy.deepcopy(DQN)
for i,j in zip(DQN.parameters(),targetDQN.parameters()):
    (j.data).copy_(i.data)
for i in targetDQN.parameters():
    i.requires_grad=False

NameError: name 'torch' is not defined

In [3]:
opt=torch.optim.Adam(DQN.parameters(),lr=0.0001)

In [5]:
#使用的算法：TD（Q-learning）、Double DQN、Multiple Step、Experience Replay
%matplotlib qt5
buffer_lengthest=8000
buffer_length_to_train=3000
minibatch_size=80
discount=0.97
env=gym.make('CartPole-v1',render_mode='rgb_array')
observation,info=env.reset()
#plt.figure('CartPole-v1',figsize=(4,4))
update_target_DQN=0
alpha_greedy=0.0001
steps=0
replay_buffer=[]
while(1):
    small_buffer=[]#创建小buffer，再整体存入replay_buffer中
    #plt.clf()#清除当前图像，必须要有，否则图片会积压住，后来很影响整体代码运行速度
    small_buffer.append(list(observation))
    #plt.imshow(env.render())#显示图像
    #plt.pause(0.00000001)#暂停一小会，必须有
    a=random.random()#使用α-greedy算法，先取一个随机数
    act=DQN(torch.from_numpy(observation))#DQN运算
    if a>alpha_greedy:#如果随机数大于阈值，选DQN分值最大的
        action=torch.argmax(act).item()
        #scoret=torch.max(act)#获得当前t时刻预测的分数
    else:#否则，随机选择动作
        action=random.randint(0,1)
        #scoret=act[action]#获得当前t时刻预测的分数
    small_buffer.append(action)
    observation, reward, terminated, truncated, info=env.step(action)#环境执行动作，获得新观测值、奖励以及其他
    small_buffer.append(reward)
    small_buffer.append(list(observation))
    steps+=1
    #下面是开始训练的代码：
    if len(replay_buffer)>=buffer_length_to_train:#当replay_buffer中的数据长度超过buffer_length_to_train的时候才开始训练
        if len(replay_buffer)==buffer_length_to_train:
            print('Start Train')
        how_many=0#在本代码算法结构中，如果在multi_step中遇到自然结束（500步），就舍弃掉这个，how_many就是来计算实际有多少个梯度，用来计算梯度平均
        opt.zero_grad()#将DQN梯度清零
        for epoch in range(minibatch_size):#建立循环，minibatch_size个
            drop_this_epoch=0#标识符，用来判断
            end_by_fail=0#标识符，用来判断
            target_score=0#TD算法的目标分数，初始化为0
            start_num=random.randint(0,len(replay_buffer)-51)#要在replay_buffer中取样本，用随机数生成第一个数的序号
            len_mult_step=random.randint(4,50)#训练使用multipe_step,也用随机数生成step距离
            for i in range(len_mult_step):
                if replay_buffer[start_num+i][4]==1:#杆子倒了，失败了
                    len_mult_step=i+1
                    end_by_fail=1#标识符
                    break
                elif replay_buffer[start_num+i][4]==2:#超过500轮，自动停止
                    drop_this_epoch=1#标识符，意味舍弃这一组数据，不计算梯度
                    break
            if drop_this_epoch==1:
                continue#如果遇到自然停止的（超过500），跳出这一轮循环，不计算梯度
            scoret=(DQN(torch.Tensor(replay_buffer[start_num][0])))[replay_buffer[start_num][1]]#获得t时刻分数
            if end_by_fail==1:#如果遇到失败的清空，target_score就是折扣汇报的累加，不需要targetDQN计算
                for i in range(len_mult_step):
                    target_score+=pow(discount,i)
            else:
                for i in range(len_mult_step):
                    target_score+=pow(discount,i)
                target_score+=torch.max(targetDQN(torch.Tensor(replay_buffer[start_num+len_mult_step-1][3])))*pow(discount,len_mult_step)
            loss=0.5*pow((scoret-target_score),2)
            loss.backward()
            how_many+=1
        for i in DQN.parameters():#loss.backward会将梯度累加，这里求梯度的平均
#             (i.grad.data).div_(how_many)
            print(i.grad)
        opt.step()#优化器执行
        #print('optim step successfully')
        update_target_DQN+=1
        if update_target_DQN==30:#更新targetDQN
            update_target_DQN=0
            for i,j in zip(DQN.parameters(),targetDQN.parameters()):
                j.data.mul_(0.3)
                j.add_((i.data)*0.7)
    ################################################
    if not (truncated or terminated):
        small_buffer.append(0)
        if len(replay_buffer)>=buffer_lengthest:
            del replay_buffer[0]#如果replay_buffer的总长度大于阈值（3000），则添加新的时候，删掉最老的（序号为0的）
        replay_buffer.append(small_buffer)
    elif terminated:
        small_buffer.append(1)
        if len(replay_buffer)>=buffer_lengthest:
            del replay_buffer[0]
        replay_buffer.append(small_buffer)
        print('Episode End , steps=', steps)
        steps=0
        print(len(replay_buffer))
        observation,info=env.reset()
    elif truncated:#超过500轮，自动停止
        small_buffer.append(2)
        if len(replay_buffer)>=buffer_lengthest:
            del replay_buffer[0]
        replay_buffer.append(small_buffer)
        print('Episode End by 500')
        steps=0
        observation,info=env.reset()

Episode End , steps= 9
9
Episode End , steps= 10
19
Episode End , steps= 9
28
Episode End , steps= 8
36
Episode End , steps= 8
44
Episode End , steps= 9
53
Episode End , steps= 9
62
Episode End , steps= 8
70
Episode End , steps= 10
80
Episode End , steps= 9
89
Episode End , steps= 9
98
Episode End , steps= 9
107
Episode End , steps= 9
116
Episode End , steps= 10
126
Episode End , steps= 8
134
Episode End , steps= 8
142
Episode End , steps= 10
152
Episode End , steps= 10
162
Episode End , steps= 10
172
Episode End , steps= 10
182
Episode End , steps= 9
191
Episode End , steps= 9
200
Episode End , steps= 9
209
Episode End , steps= 8
217
Episode End , steps= 10
227
Episode End , steps= 9
236
Episode End , steps= 9
245
Episode End , steps= 10
255
Episode End , steps= 10
265
Episode End , steps= 10
275
Episode End , steps= 10
285
Episode End , steps= 10
295
Episode End , steps= 10
305
Episode End , steps= 8
313
Episode End , steps= 9
322
Episode End , steps= 9
331
Episode End , steps= 10
34

Episode End , steps= 10
2933
Episode End , steps= 9
2942
Episode End , steps= 10
2952
Episode End , steps= 9
2961
Episode End , steps= 10
2971
Episode End , steps= 10
2981
Episode End , steps= 9
2990
Episode End , steps= 8
2998
Start Train
tensor([[-8.2541e-06,  1.4605e-05,  1.7658e-06, -2.5974e-05],
        [ 2.9486e-05,  7.4512e-04, -3.9986e-05, -1.1339e-03],
        [-1.0580e-05,  1.9448e-05,  1.4302e-05, -3.8666e-05],
        [-1.1913e-04, -2.9256e-03,  1.7254e-04,  4.4483e-03],
        [ 2.5785e-05,  5.6529e-04, -2.3476e-05, -8.9210e-04],
        [ 4.0223e-05,  1.2072e-03, -7.1443e-05, -1.8464e-03],
        [-1.6750e-04, -4.1194e-03,  2.4498e-04,  6.2520e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-3.3460e-06, -5.5398e-05,  7.8992e-06,  9.1546e-05],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 6.6456e-07, -2.8304e-04,  1.8441e-05,  4.3381e-04],
        [ 0.0000

tensor([[ 4.4453e-06, -4.2918e-05, -3.8362e-06,  1.0595e-04],
        [-2.9625e-05, -3.5905e-04,  5.0672e-05,  5.7192e-04],
        [-5.2860e-05, -6.5642e-04,  6.9590e-05,  1.0062e-03],
        [-3.9900e-05, -2.6525e-03,  7.4309e-05,  4.0412e-03],
        [-1.6856e-05,  4.5675e-04,  2.0350e-05, -7.2240e-04],
        [ 3.6123e-06,  7.5137e-04, -4.0973e-06, -1.1438e-03],
        [-1.4365e-04, -5.6273e-03,  2.2233e-04,  8.5889e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-3.4020e-05, -5.9222e-04,  5.2694e-05,  9.1441e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-2.0044e-06, -2.0204e-04,  4.3715e-07,  2.9649e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-1.0554e-04, -3.5587e-03,  1.5959e-04,  5.4594e-03],
        [-1.5020e-04, -4.6976e-03,  2.2622e-04,  7.1613e-03],
        [-7.2697e-05, -1.5298e-03,  1.0300e-04,  2.3325e-03],
        

tensor([[-4.4630e-06, -1.5012e-04, -1.4630e-05,  2.1285e-04],
        [-1.3625e-04, -2.8024e-03,  1.7727e-04,  4.2814e-03],
        [-1.0240e-04, -2.0916e-03,  1.3405e-04,  3.2029e-03],
        [-7.4492e-05, -2.9720e-03,  1.0427e-04,  4.5591e-03],
        [-2.0934e-06,  6.4167e-04,  2.1063e-05, -9.3969e-04],
        [ 1.5277e-05,  6.5735e-04, -1.0383e-05, -9.8368e-04],
        [-2.3372e-04, -6.9000e-03,  2.9007e-04,  1.0519e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 5.4791e-07,  5.2380e-06, -4.8272e-07, -7.7752e-06],
        [-5.5378e-05, -7.8552e-04,  7.0603e-05,  1.2005e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-1.0290e-06, -4.3701e-04, -1.5716e-06,  6.6076e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-2.1434e-04, -5.4007e-03,  2.6567e-04,  8.2277e-03],
        [-3.0695e-04, -7.6027e-03,  3.9936e-04,  1.1635e-02],
        [-1.0178e-04, -1.8468e-03,  1.3171e-04,  2.8214e-03],
        

tensor([[ 2.0870e-05, -1.2640e-04,  3.8141e-06,  1.6338e-04],
        [-1.6199e-04, -4.3197e-03,  2.9602e-04,  6.6195e-03],
        [-1.3859e-04, -3.0653e-03,  2.0351e-04,  4.7066e-03],
        [-4.6419e-05, -2.4801e-03,  1.3559e-04,  3.7782e-03],
        [-2.7072e-05,  6.4182e-04, -2.2011e-05, -9.1434e-04],
        [-9.8897e-06,  3.7801e-04, -1.6331e-05, -5.4883e-04],
        [-1.8648e-04, -6.6928e-03,  4.0444e-04,  1.0194e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-5.1504e-05, -8.4134e-04,  7.3628e-05,  1.2998e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-8.2317e-06, -5.1799e-04,  3.6045e-05,  7.7044e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-1.9101e-04, -5.6530e-03,  3.6214e-04,  8.6331e-03],
        [-3.0337e-04, -8.3009e-03,  5.3629e-04,  1.2730e-02],
        [-1.1903e-04, -2.7539e-03,  1.8164e-04,  4.2366e-03],
        

...

tensor([[-8.3642e-06, -3.3819e-04, -8.7043e-06,  5.0365e-04],
        [-3.5709e-04, -8.2358e-03,  5.0907e-04,  1.2569e-02],
        [-2.6163e-04, -5.7209e-03,  3.8626e-04,  8.7304e-03],
        [ 4.0752e-06, -1.2127e-03,  9.2150e-06,  1.8380e-03],
        [ 8.9461e-06,  2.4793e-04,  5.0172e-06, -3.7104e-04],
        [-9.8351e-05, -2.2442e-03,  1.5003e-04,  3.4340e-03],
        [-3.0856e-04, -9.0013e-03,  4.5033e-04,  1.3718e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-1.2004e-04, -2.0239e-03,  1.6474e-04,  3.0859e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-4.7887e-06, -4.8192e-04,  1.3040e-05,  7.4810e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-3.9553e-04, -9.4558e-03,  5.6246e-04,  1.4413e-02],
        [-5.4256e-04, -1.3168e-02,  7.8039e-04,  2.0091e-02],
        [-2.4125e-04, -5.1996e-03,  3.5508e-04,  7.9244e-03],
        

tensor([[-5.5166e-05, -5.1756e-04, -9.6247e-06,  7.9299e-04],
        [-4.8122e-04, -1.0093e-02,  5.9947e-04,  1.5560e-02],
        [-3.0036e-04, -6.5998e-03,  4.2439e-04,  1.0149e-02],
        [ 1.2193e-05, -5.5310e-04, -4.4762e-05,  8.8778e-04],
        [ 1.7098e-05,  2.6440e-04,  5.8960e-06, -4.1209e-04],
        [-1.2616e-04, -2.8947e-03,  1.7178e-04,  4.4239e-03],
        [-4.5526e-04, -1.0930e-02,  4.9637e-04,  1.6803e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-1.5902e-04, -2.6795e-03,  2.1915e-04,  4.1111e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-5.8007e-05, -6.9091e-04,  1.2122e-05,  1.0773e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-5.0739e-04, -1.1347e-02,  6.3877e-04,  1.7452e-02],
        [-6.8806e-04, -1.5184e-02,  8.4965e-04,  2.3380e-02],
        [-2.6639e-04, -6.1175e-03,  4.2986e-04,  9.4243e-03],
        

tensor([[ 7.6394e-06, -2.5160e-04,  2.2961e-06,  4.1387e-04],
        [-3.4729e-04, -1.0810e-02,  6.8559e-04,  1.6723e-02],
        [-3.2072e-04, -7.9148e-03,  5.3155e-04,  1.2204e-02],
        [ 1.1223e-04,  5.4078e-04, -1.0812e-04, -8.1146e-04],
        [-1.8037e-05,  3.4553e-04,  8.4566e-06, -5.2213e-04],
        [-1.7461e-04, -3.5222e-03,  2.6927e-04,  5.4409e-03],
        [-2.6442e-04, -1.1246e-02,  5.9502e-04,  1.7424e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 4.2283e-06,  4.6913e-05, -4.9507e-06, -7.0709e-05],
        [-1.7229e-04, -2.9603e-03,  2.7206e-04,  4.5562e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 1.3105e-05, -3.3353e-04,  8.2495e-06,  5.5696e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-3.6891e-04, -1.1494e-02,  6.8895e-04,  1.7745e-02],
        [-5.3277e-04, -1.6089e-02,  1.0250e-03,  2.4926e-02],
        [-2.7476e-04, -6.6814e-03,  4.5393e-04,  1.0244e-02],
        

tensor([[-8.4757e-06, -6.5866e-04, -1.0493e-05,  1.1102e-03],
        [-5.8881e-04, -1.3601e-02,  7.8777e-04,  2.0999e-02],
        [-5.0734e-04, -1.0555e-02,  6.6022e-04,  1.6215e-02],
        [ 1.3158e-04,  1.6687e-03, -1.6730e-04, -2.5032e-03],
        [ 9.2827e-07,  3.2159e-04, -1.1011e-06, -5.1768e-04],
        [-2.6754e-04, -5.2889e-03,  3.4919e-04,  8.0801e-03],
        [-5.9424e-04, -1.5111e-02,  7.4268e-04,  2.3357e-02],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-2.3981e-04, -4.2034e-03,  3.4100e-04,  6.4212e-03],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 8.0935e-06, -5.6525e-04,  3.1728e-06,  9.5647e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [-6.4109e-04, -1.4740e-02,  8.1418e-04,  2.2689e-02],
        [-9.5144e-04, -2.0993e-02,  1.2704e-03,  3.2397e-02],
        [-3.8923e-04, -7.9279e-03,  5.1257e-04,  1.2139e-02],
        

KeyboardInterrupt: 

In [122]:
torch.save(DQN.state_dict(),'./DDQN_CartPole_state_dict.pth')

In [123]:
torch.save(DQN,'./DDQN_CartPole_full.pth')

In [120]:
%matplotlib qt5
env=gym.make('CartPole-v1',render_mode='rgb_array')
observation, info=env.reset()
plt.figure('CartPole-v1',figsize=(4,4))
for t in range(1000):
    plt.clf()
    plt.imshow(env.render())
    plt.pause(0.00001)
    #print(observation)
    with torch.no_grad():
        action=torch.argmax(DQN(torch.from_numpy(observation))).item()
    #print(aa.type())
    #b=DQN(aa)
    #print(b)
    #print(torch.from_numpy(observation))
    #action=env.action_space.sample()
    #print(action)
    observation, reward, terminated, truncated, info=env.step(action)
    #print(observation.type())
    if truncated:
        print('Finished by 500')
        break
    elif terminated:
        print('Finished')
        break
env.close()

Finished by 500
